In [21]:
import pandas as pd
import numpy as np
import time
from datetime import datetime, timedelta
import requests as r
from http.client import IncompleteRead
import nfl_data_py as nfl

In [22]:
#If the table reached column limit, use below
# pd.options.display.max_columns = 500


print (pd.options.display.max_columns) # <--- this will display your limit 

20


In [23]:
#We'll get actual performance data from nfl data py (Python vesion of nflfastR) 
#Github repo: https://github.com/cooperdff/nfl_data_py

# Performance Data

In [24]:
#Rather than manually change the date range to add new seasons, we can add a rule
#This will simply check if today is before or after sept 1st
#If it's before, we know the max season is the current year
#Otherwise, it's the current year plus 1

#So on Jan 1st 2025, the rule will ensure that we're still pulling for the 2024 season
#And on Sept 1st 2025, the "max year" or the end of the range should be 2026

if datetime.today().month >= 9:
    max_year = datetime.today().year + 1
else:
    max_year = datetime.today().year

In [34]:
max_year

2024

In [35]:
if datetime.today().month >= 2:
    max_year = datetime.today().year + 1
else:
    max_year = datetime.today().year

In [36]:
max_year

2025

In [33]:
years = [i for i in range(2018, max_year)]

In [27]:
weekly_data = nfl.import_weekly_data(years)

Downcasting floats.


In [28]:
#Let's filter for only regular season

In [29]:
weekly_data = weekly_data[weekly_data['season_type'] == 'REG'].reset_index(drop=True)

In [30]:
weekly_data.head()

,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,...,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr
0,00-0000003,None,Abdul-Karim al-Jabbar,RB,RB,None,MIA,1999,1,REG,...,0.0,0.292378,0,0.0,0.052632,NaN,NaN,0.0,12.7,13.7
1,00-0000003,None,Abdul-Karim al-Jabbar,RB,RB,None,MIA,1999,2,REG,...,1.0,0.377009,0,0.0,0.117647,NaN,NaN,0.0,5.1,8.1
2,00-0000003,None,Abdul-Karim al-Jabbar,RB,RB,None,MIA,1999,4,REG,...,0.0,-0.699578,0,NaN,0.023810,NaN,NaN,0.0,0.2,0.2
3,00-0000003,None,Abdul-Karim al-Jabbar,RB,RB,None,CLE,1999,7,REG,...,0.0,-0.228454,0,0.0,0.050000,NaN,NaN,0.0,3.5,5.5
4,00-0000003,None,Abdul-Karim al-Jabbar,RB,RB,None,CLE,1999,8,REG,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,3.9,3.9


In [31]:
#I want to organize these columns based on the team that's listed and their opponent
#For the opponent, I want to capture these columns as defensive stats (yards allowed, etc)
#The pfr table has opponent as a column, so let's use that

In [32]:
weekly_pfr = nfl.import_weekly_pfr('pass',years)

ValueError: Data not available before 2018.

In [ ]:
weekly_pfr = weekly_pfr[weekly_pfr['game_type'] == 'REG'].reset_index(drop=True)

In [ ]:
weekly_pfr.head()

In [ ]:
team_opp = weekly_pfr[['season', 'week', 'team', 'opponent']]

In [ ]:
team_opp[team_opp['team'] == 'OAK'].head()

In [ ]:
#Due to raiders moving, let's keep all raider rows as LV

In [ ]:
team_opp.loc[team_opp['team'] == 'OAK', 'team'] = 'LV'
team_opp.loc[team_opp['opponent'] == 'OAK', 'opponent'] = 'LV'

In [ ]:
#Table is based on passing, so sometimes there's two QBs, so we need to drop duplicates as we just want opponent

In [ ]:
team_opp = team_opp.drop_duplicates()

In [ ]:
len(weekly_data)

In [ ]:
len(team_opp)

In [ ]:
#Lets add an opponent column to the 
weekly_data = weekly_data.merge(team_opp, left_on = ['recent_team', 'season', 'week'],
                                right_on = ['team', 'season', 'week'],
                                how='left'
                               )

In [ ]:
len(weekly_data)

In [ ]:
#Confirming we don't have null opponent names?
weekly_data[weekly_data['opponent'].isna()].head()

In [ ]:
#Definitions for columns can be found here: https://www.nflfastr.com/reference/calculate_player_stats.html

In [ ]:
#Lets see what columns we have

In [ ]:
[f"{value}, {i}" for value, i in enumerate(nfl.see_weekly_cols())]

In [ ]:
#We can group these by team by week to get either the sum or avg of certain columns
#Group by team by week b/c this is at the player level

In [ ]:
weekly_agg = weekly_data.groupby(['season', 'week', 'recent_team']).agg({'completions':'sum',
                                                              'attempts':'sum',
                                                              'passing_yards':'sum' ,
                                                             'passing_tds':'sum', 
                                                             'interceptions':'sum',
                                                             'sacks':'sum',
                                                             'sack_yards':'sum',
                                                             'sack_fumbles':'sum',
                                                             'sack_fumbles_lost':'sum',
                                                             'passing_air_yards':'sum',
                                                             'passing_yards_after_catch':'sum',
                                                             'passing_first_downs':'sum',
                                                             'passing_epa':['mean','sum'],
                                                             'passing_2pt_conversions':'sum',
                                                             'dakota':'mean',
                                                             'carries':'sum',
                                                             'rushing_yards':'sum',
                                                             'rushing_tds':'sum',
                                                             'rushing_fumbles':'sum',
                                                             'rushing_fumbles_lost':'sum',
                                                             'rushing_first_downs':'sum',
                                                             'rushing_epa':['mean','sum'],
                                                             'rushing_2pt_conversions':'sum',
                                                             'receptions':'sum',
                                                             'targets':'sum',
                                                             'receiving_yards':'sum',
                                                             'receiving_tds':'sum',
                                                             'receiving_fumbles':'sum',
                                                             'receiving_fumbles_lost':'sum',
                                                             'receiving_air_yards':'sum',
                                                             'receiving_yards_after_catch':'sum',
                                                             'receiving_first_downs':'sum',
                                                             'receiving_epa':['mean','sum'],
                                                             'receiving_2pt_conversions':'sum',
                                                             'wopr':['mean','sum'],
                                                             'special_teams_tds':'sum',
                                                             'fantasy_points':'sum',
                                                             'fantasy_points_ppr':'sum'
                                                              }).reset_index()

In [ ]:
weekly_agg = weekly_agg.droplevel(1,axis=1)

In [ ]:
#Columns to re-create with game totals:
#PACR: Passing yards / Passing air yards
#RACR: Receiving yards / Receiving air yards

In [ ]:
weekly_agg['pacr'] = weekly_agg['passing_yards'] / weekly_agg['passing_air_yards']

In [ ]:
weekly_agg['racr'] = weekly_agg['receiving_yards'] / weekly_agg['receiving_air_yards']

In [ ]:
weekly_agg.index.is_unique

In [ ]:
#Since we dropped the multi-level index, some columns have the same name b/c it's referring mean or sum
#Manually renaming them because identifying by name will change all columns with the same name

In [ ]:
weekly_agg.columns

In [ ]:
weekly_agg.columns = ['season','week', 'team', 'completions', 'attempts', 'passing_yards',
       'passing_tds', 'interceptions', 'sacks', 'sack_yards', 'sack_fumbles',
       'sack_fumbles_lost', 'passing_air_yards', 'passing_yards_after_catch',
       'passing_first_downs', 'avg_passing_epa', 'total_passing_epa', 'passing_2pt_conversions', 
        'dakota', 'carries', 'rushing_yards', 'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'avg_rushing_epa', 'total_rushing_epa','rushing_2pt_conversions',
        'receptions', 'targets', 'receiving_yards',
       'receiving_tds', 'receiving_fumbles', 'receiving_fumbles_lost',
       'receiving_air_yards', 'receiving_yards_after_catch',
       'receiving_first_downs', 'avg_receiving_epa', 'total_receiving_epa',
       'receiving_2pt_conversions', 'avg_wopr', 'total_wopr', 'special_teams_tds',
       'fantasy_points', 'fantasy_points_ppr', 'pacr', 'racr']

In [ ]:
weekly_agg.columns

In [ ]:
weekly_agg.head()

In [ ]:
#This dataset doesn't have defensive stats
#But if we group by opponent, we can get the same stats but as "allowed"
#For example, bucs v dal week 1 2021, bucs passing yards = dal passing yards allowed

In [ ]:
weekly_agg_def = weekly_data.groupby(['season', 'week', 'opponent']).agg({'completions':'sum',
                                                              'attempts':'sum',
                                                              'passing_yards':'sum' ,
                                                             'passing_tds':'sum', 
                                                             'interceptions':'sum',
                                                             'sacks':'sum',
                                                             'sack_yards':'sum',
                                                             'sack_fumbles':'sum',
                                                             'sack_fumbles_lost':'sum',
                                                             'passing_air_yards':'sum',
                                                             'passing_yards_after_catch':'sum',
                                                             'passing_first_downs':'sum',
                                                             'passing_epa':['mean','sum'],
                                                             'passing_2pt_conversions':'sum',
                                                             'dakota':'mean',
                                                             'carries':'sum',
                                                             'rushing_yards':'sum',
                                                             'rushing_tds':'sum',
                                                             'rushing_fumbles':'sum',
                                                             'rushing_fumbles_lost':'sum',
                                                             'rushing_first_downs':'sum',
                                                             'rushing_epa':['mean','sum'],
                                                             'rushing_2pt_conversions':'sum',
                                                             'receptions':'sum',
                                                             'targets':'sum',
                                                             'receiving_yards':'sum',
                                                             'receiving_tds':'sum',
                                                             'receiving_fumbles':'sum',
                                                             'receiving_fumbles_lost':'sum',
                                                             'receiving_air_yards':'sum',
                                                             'receiving_yards_after_catch':'sum',
                                                             'receiving_first_downs':'sum',
                                                             'receiving_epa':['mean','sum'],
                                                             'receiving_2pt_conversions':'sum',
                                                             'wopr':['mean','sum'],
                                                             'special_teams_tds':'sum',
                                                             'fantasy_points':'sum',
                                                             'fantasy_points_ppr':'sum'
                                                              }).reset_index()

In [ ]:
weekly_agg_def = weekly_agg_def.droplevel(1,axis=1)

In [ ]:
weekly_agg_def['pacr'] = weekly_agg_def['passing_yards'] / weekly_agg_def['passing_air_yards']

In [ ]:
weekly_agg_def['racr'] = weekly_agg_def['receiving_yards'] / weekly_agg_def['receiving_air_yards']

In [ ]:
#Since this is defensive allowed, we can take the columns from weekly_agg and add _allowed
weekly_agg_def.columns = [i+'_allowed' if i not in ['season', 'week', 'team', 'opponent'] else i for i in weekly_agg.columns]

In [ ]:
weekly_agg_def.columns

In [ ]:
weekly_agg_def.head()

In [ ]:
#Sort the table by year, team, and week

In [ ]:
weekly_agg = weekly_agg.sort_values(by=['season','team','week']).reset_index(drop=True)
weekly_agg_def = weekly_agg_def.sort_values(by=['season','team','week']).reset_index(drop=True)

In [ ]:
#Now let's join the two tables so we have both offense and defensive stats

In [ ]:
weekly_agg = weekly_agg.merge(weekly_agg_def, on=['season','team','week'])

In [ ]:
weekly_agg.head()

In [ ]:
#Lets get rolling values for these columns

In [ ]:
original_week = weekly_agg['week'].copy()

In [ ]:
weekly_agg_rolling = weekly_agg.groupby(['season', 'team'])[
    [i for i in weekly_agg.columns if i not in ['season', 'week', 'team']]
].rolling(3).mean().reset_index()

In [ ]:
weekly_agg_rolling = weekly_agg_rolling.drop('level_2', axis=1)

In [ ]:
weekly_agg_rolling['week'] = original_week

In [ ]:
#Confirming the rolling is working as expected

In [ ]:
weekly_agg_rolling.iloc[14:25, :]

In [ ]:
weekly_agg_rolling.columns = [i+'_rolling' if i not in ['season', 'week', 'team', 'opponent'] \
                              else i for i in weekly_agg_rolling.columns]

In [ ]:
weekly_agg_rolling.head()

In [ ]:
#Lets join the two tables

In [ ]:
len(weekly_agg) == len(weekly_agg_rolling)

In [ ]:
len(weekly_agg)

In [ ]:
weekly_agg = weekly_agg.merge(weekly_agg_rolling, on=['season', 'week', 'team'])

In [ ]:
len(weekly_agg)

In [ ]:
weekly_agg.head()

In [ ]:
#Lets get a rank of all our metrics

In [ ]:
columns_to_rank = [i for i in weekly_agg.columns if i not in ['season', 'week', 'team', 'opponent']]

In [ ]:
for col in columns_to_rank:
    rank_col_name = f'rank_{col}_weekly'
    weekly_agg[rank_col_name] = weekly_agg.groupby(['season', 'week'])[col].rank(ascending=False)

In [ ]:
weekly_agg.head()

In [ ]:
#Let's rank columns based on what was done earlier in season

In [ ]:
#First we get a sum or avg of these columns by season up to that point in the week

In [ ]:
#First we'll get the columns that should be summed up (non-avg columns)

In [ ]:
columns_to_sum = [i for i in weekly_agg.columns \
                  if i not in ['season', 'week', 'team','avg_passing_epa', 'dakota', 'avg_rushing_epa', 
                  'avg_receiving_epa', 'avg_wopr', 'pacr', 'racr',
                              'avg_passing_epa_allowed', 'dakota_allowed', 'avg_rushing_epa_allowed', 
                  'avg_receiving_epa_allowed', 'avg_wopr_allowed', 'pacr_allowed', 'racr_allowed']
                 and '_rolling' not in i
                 and 'rank' not in i]

In [ ]:
columns_to_sum

In [ ]:
for col in columns_to_sum:
    expand_col_name = f'expand_{col}'
    weekly_agg[expand_col_name] = weekly_agg.sort_values(by='week').groupby(['season', 'team'])[col].cumsum()

In [ ]:
weekly_agg.head()

In [ ]:
#Lets check this is working

In [ ]:
weekly_agg[['season', 'week', 'team','completions', 'expand_completions']].iloc[:25, :]

In [ ]:
#Great, lets do this for averages

In [ ]:
colums_to_expand_avg = ['avg_passing_epa',
                       'dakota',
                       'avg_rushing_epa',
                       'avg_receiving_epa',
                       'avg_wopr',
                       'pacr', 
                       'racr',
                       'avg_passing_epa_allowed', 'dakota_allowed', 'avg_rushing_epa_allowed', 
                  'avg_receiving_epa_allowed', 'avg_wopr_allowed', 'pacr_allowed', 'racr_allowed']

In [ ]:
for col in colums_to_expand_avg:
    expand_col_name = f'expand_{col}'
    cumulative_sum = weekly_agg.sort_values(by='week').groupby(['season', 'team'])[col].cumsum()
    cumulative_count = weekly_agg.sort_values(by='week').groupby(['season', 'team'])[col].cumcount() + 1
    weekly_agg[expand_col_name] = cumulative_sum / cumulative_count

In [ ]:
#Lets confirm this worked

In [ ]:
weekly_agg[['season', 'week', 'team','pacr', 'expand_pacr']].iloc[:25, :]

In [ ]:
#Now we'll get the ranking for these "expanded" columns

In [ ]:
for col in weekly_agg.columns:
    if 'expand_' in col:
        rank_col_name = f'rank_{col}_weekly'
        weekly_agg[rank_col_name] = weekly_agg.groupby(['season', 'week'])[col].rank(ascending=False)

In [ ]:
#Now we have to shift our data down a row
#When we leverage the data, we won't have the current week's information
#We'll only have the avg as of the prior week
#So we need the avgs, totals, values, etc to reflect what has occured up to that point

In [ ]:
weekly_agg.head()

In [ ]:
columns_to_shift = [i for i in weekly_agg.columns if i not in ['season', 'week', 'team']]

In [ ]:
for i in columns_to_shift:
    weekly_agg[i+'_shifted'] = weekly_agg.groupby(['season', 'team'])[i].shift(periods=1)

In [ ]:
#Check the shift worked

In [ ]:
weekly_agg[['season', 'week', 'team', 
            'completions', 'completions_shifted','rank_completions_weekly', 'rank_completions_weekly_shifted',
            'completions_rolling', 'completions_rolling_shifted']].head(20)

In [ ]:
#This is working as expected
#We're keeping the shifted columns since that's what we need for the model

In [ ]:
weekly_agg = weekly_agg[[i for i in weekly_agg.columns if '_shifted' in i or i in ['season', 'week', 'team'] ]]

In [ ]:
weekly_agg.head()

# Passing Data

In [ ]:
weekly_pfr = nfl.import_weekly_pfr('pass',years)

In [ ]:
weekly_pfr = weekly_pfr[weekly_pfr['game_type'] == 'REG'].reset_index(drop=True)

In [ ]:
weekly_pfr.head()

In [ ]:
weekly_pfr.columns

In [ ]:
#Example/Definitions available here:
#https://www.pro-football-reference.com/players/M/MahoPa00/gamelog/2022/advanced/#all_advanced_passing
#Hover over column names to get definitions

In [ ]:
#Noticed that the columns like def_times_blitzed were null, are they always null?

In [ ]:
weekly_pfr[~weekly_pfr['def_times_blitzed'].isna()].head()

In [ ]:
weekly_pfr[~weekly_pfr['def_times_hurried'].isna()].head()

In [ ]:
weekly_pfr[~weekly_pfr['def_times_hitqb'].isna()].head()

In [ ]:
#Odd that these columns are all null

In [ ]:
#Also noticed there's no receiving drops

In [ ]:
weekly_pfr['receiving_drop'].unique()

In [ ]:
weekly_pfr['receiving_drop_pct'].unique()

In [ ]:
#We'll just drop them

In [ ]:
weekly_pfr = weekly_pfr.drop(['def_times_hurried', 
                              'def_times_hitqb', 
                             'def_times_blitzed',
                              'receiving_drop',
                             'receiving_drop_pct'], axis=1)

In [ ]:
#Changing raiders to LV b/c of their move

In [ ]:
weekly_pfr.loc[weekly_pfr['team'] == 'OAK', 'team'] = 'LV'
weekly_pfr.loc[weekly_pfr['opponent'] == 'OAK', 'opponent'] = 'LV'

In [ ]:
#Lets aggregate across week and season

In [ ]:
pfr_agg = weekly_pfr.groupby(['season', 'week', 'team']).agg({'passing_drops':'sum',
                                                              'passing_drop_pct':'mean', 
                                                              'passing_bad_throws':'sum', 
                                                              'passing_bad_throw_pct':'mean',
                                                              'times_sacked':'sum',
                                                              'times_blitzed':'sum', 
                                                              'times_hurried':'sum',
                                                              'times_hit':'sum',
                                                              'times_pressured':'sum',
                                                              'times_pressured_pct':'mean'}).reset_index()

In [ ]:
pfr_agg.head()

In [ ]:
#Like before, let's group by opponent to get "allowed" columns
#We'll use this as defensive stats

In [ ]:
pfr_agg_def = weekly_pfr.groupby(['season', 'week', 'opponent']).agg({'passing_drops':'sum',
                                                              'passing_drop_pct':'mean', 
                                                              'passing_bad_throws':'sum', 
                                                              'passing_bad_throw_pct':'mean',
                                                              'times_sacked':'sum',
                                                              'times_blitzed':'sum', 
                                                              'times_hurried':'sum',
                                                              'times_hit':'sum',
                                                              'times_pressured':'sum',
                                                              'times_pressured_pct':'mean'}).reset_index()

In [ ]:
pfr_agg_def.head()

In [ ]:
pfr_agg_def.columns = [i+'_allowed' if i not in ['season', 'team', 'week'] else i for i in pfr_agg.columns ]

In [ ]:
#Lets join our offensive and defensive stats

In [ ]:
pfr_agg = pfr_agg.sort_values(by=['season','team','week']).reset_index(drop=True)
pfr_agg_def = pfr_agg_def.sort_values(by=['season','team','week']).reset_index(drop=True)

In [ ]:
len(pfr_agg)

In [ ]:
len(pfr_agg) == len(pfr_agg_def)

In [ ]:
pfr_agg = pfr_agg.merge(pfr_agg_def, on=['season','team','week'])

In [ ]:
pfr_agg.head()

In [ ]:
#Now lets get rolling columns

In [ ]:
pfr_agg = pfr_agg.sort_values(by=['season','team','week']).reset_index(drop=True)

In [ ]:
original_week = pfr_agg['week'].copy()

In [ ]:
pfr_agg_rolling = pfr_agg.groupby(['season', 'team'])[['passing_drops', 'passing_drop_pct',
                                                       'passing_bad_throws',
                                                       'passing_bad_throw_pct', 
                                                       'times_sacked', 
                                                       'times_blitzed',
       'times_hurried', 'times_hit', 'times_pressured', 'times_pressured_pct']].rolling(3).mean().reset_index()

In [ ]:
pfr_agg_rolling = pfr_agg_rolling.drop('level_2', axis=1)

In [ ]:
pfr_agg_rolling['week'] = original_week

In [ ]:
pfr_agg_rolling.head()

In [ ]:
pfr_agg_rolling.columns = [i+'_rolling' if i not in ['season', 'week', 'team', 'opponent'] \
                              else i for i in pfr_agg_rolling.columns]

In [ ]:
pfr_agg_rolling.columns

In [ ]:
#Joining rolling to regular table

In [ ]:
len(pfr_agg) == len(pfr_agg_rolling)

In [ ]:
pfr_agg = pfr_agg.merge(pfr_agg_rolling, on=['season', 'team', 'week'])

In [ ]:
len(pfr_agg) == len(pfr_agg_rolling)

In [ ]:
pfr_agg.columns

In [ ]:
#Lets get the ranks for each week for these metrics

In [ ]:
columns_to_rank = [i for i in pfr_agg.columns if i not in ['season', 'week', 'team']]

In [ ]:
for col in columns_to_rank:
    rank_col_name = f'rank_{col}_weekly'
    pfr_agg[rank_col_name] = pfr_agg.groupby(['season', 'week'])[col].rank(ascending=False)

In [ ]:
pfr_agg.head()

In [ ]:
#Lets get sums/avg up to each part of the season

In [ ]:
colums_to_expand_sum = ['passing_drops', 
       'passing_bad_throws', 'times_sacked',
       'times_blitzed', 'times_hurried', 
                        'times_hit', 'times_pressured',
        'passing_drops_allowed', 
       'passing_bad_throws_allowed', 'times_sacked_allowed',
       'times_blitzed_allowed', 'times_hurried_allowed', 
                        'times_hit_allowed', 'times_pressured_allowed']

In [ ]:
for col in colums_to_expand_sum:
    expand_col_name = f'expand_{col}'
    pfr_agg[expand_col_name] = pfr_agg.sort_values(by='week').groupby(['season', 'team'])[col].cumsum()

In [ ]:
pfr_agg.head()

In [ ]:
#Lets check this is working

In [ ]:
pfr_agg[['season', 'week', 'team','passing_drops', 'expand_passing_drops']].iloc[:25, :]

In [ ]:
#Great, lets do this for averages

In [ ]:
colums_to_expand_avg = ['passing_drop_pct','passing_bad_throw_pct','times_pressured_pct',
                       'passing_drop_pct_allowed','passing_bad_throw_pct_allowed','times_pressured_pct_allowed']

In [ ]:
for col in colums_to_expand_avg:
    expand_col_name = f'expand_{col}'
    cumulative_sum = pfr_agg.sort_values(by='week').groupby(['season', 'team'])[col].cumsum()
    cumulative_count = pfr_agg.sort_values(by='week').groupby(['season', 'team'])[col].cumcount() + 1
    pfr_agg[expand_col_name] = cumulative_sum / cumulative_count

In [ ]:
#Lets confirm this worked

In [ ]:
pfr_agg[['season', 'week', 'team','passing_drop_pct', 'expand_passing_drop_pct']].iloc[:25, :]

In [ ]:
#Now we'll get the rank of the expanding columns

In [ ]:
for col in pfr_agg.columns:
    if 'expand_' in col:
        rank_col_name = f'rank_{col}_weekly'
        pfr_agg[rank_col_name] = pfr_agg.groupby(['season', 'week'])[col].rank(ascending=False)

In [ ]:
#Now to shift the data

In [ ]:
columns_to_shift = [i for i in pfr_agg.columns if i not in ['season', 'week', 'team']]

In [ ]:
for i in columns_to_shift:
    pfr_agg[i+'_shifted'] = pfr_agg.groupby(['season', 'team'])[i].shift(periods=1)

In [ ]:
pfr_agg.head()

In [ ]:
#Lets confirm this worked

In [ ]:
pfr_agg[['season', 'team', 'week','passing_drops', 'passing_drops_shifted',
        'passing_drops_rolling', 'passing_drops_rolling_shifted',
        'rank_passing_drops_weekly', 'rank_passing_drops_weekly_shifted']].iloc[:25, :]

In [ ]:
#Keep the shifted columns

In [ ]:
pfr_agg = pfr_agg[[i for i in pfr_agg.columns if '_shifted' in i or i in ['season', 'week', 'team'] ]]

In [ ]:
#Now let's join to weekly agg

In [ ]:
len(weekly_agg)

In [ ]:
len(weekly_agg) == len(pfr_agg)

In [ ]:
weekly_agg = weekly_agg.merge(pfr_agg, on=['season', 'team', 'week'])

In [ ]:
len(weekly_agg)

In [ ]:
weekly_agg.head()

# Rush data

In [ ]:
weekly_pfr_rush = nfl.import_weekly_pfr('rush',years)

In [ ]:
weekly_pfr_rush = weekly_pfr_rush[weekly_pfr_rush['game_type'] == 'REG'].reset_index(drop=True)

In [ ]:
weekly_pfr_rush.head()

In [ ]:
#I see null for receiving_broken_tackles, is this the case for all teams?

In [ ]:
weekly_pfr_rush[~weekly_pfr_rush['receiving_broken_tackles'].isna()].head()

In [ ]:
#We don't need that column

In [ ]:
weekly_pfr_rush =  weekly_pfr_rush.drop('receiving_broken_tackles', axis=1)

In [ ]:
weekly_pfr_rush.head()

In [ ]:
weekly_pfr_rush.loc[weekly_pfr_rush['team'] == 'OAK', 'team'] = 'LV'
weekly_pfr_rush.loc[weekly_pfr_rush['opponent'] == 'OAK', 'opponent'] = 'LV'

In [ ]:
#Lets get our aggreate values for rushing
#The avg columns we can calculate once aggregated

In [ ]:
weekly_pfr_rush.columns

In [ ]:
pfr_agg_rush = weekly_pfr_rush.groupby(['season', 'week', 'team'])[['carries',
                                                                   'rushing_yards_before_contact',
                                                                   'rushing_yards_after_contact',
                                                                   'rushing_broken_tackles']].sum().reset_index()

In [ ]:
pfr_agg_rush.head()

In [ ]:
#I checked PFR to confirm that rushing_yards_before_contact + rushing_yards_after_contact = total rush yards

In [ ]:
#Lets get the avg yards before and after contact

In [ ]:
#Let's get "allowed columns"

In [ ]:
pfr_agg_rush_def = weekly_pfr_rush.groupby(['season', 'week', 'opponent'])[['carries',
                                                                   'rushing_yards_before_contact',
                                                                   'rushing_yards_after_contact',
                                                                   'rushing_broken_tackles']].sum().reset_index()

In [ ]:
#Let's change the column names

In [ ]:
pfr_agg_rush.columns

In [ ]:
pfr_agg_rush_def.columns = [i+'_allowed' if i not in ['season', 'team','week', 'opponent'] \
                            else i for i in pfr_agg_rush.columns]

In [ ]:
pfr_agg_rush_def.head()

In [ ]:
#Lets join the rushing data with the rushing allowed data

In [ ]:
pfr_agg_rush = pfr_agg_rush.sort_values(by=['season','team','week']).reset_index(drop=True)
pfr_agg_rush_def = pfr_agg_rush_def.sort_values(by=['season','team','week']).reset_index(drop=True)

In [ ]:
len(pfr_agg_rush)

In [ ]:
len(pfr_agg_rush) == len(pfr_agg_rush_def)

In [ ]:
pfr_agg_rush = pfr_agg_rush.merge(pfr_agg_rush_def, on=['season', 'week', 'team'])

In [ ]:
#Lets get the avg yards before and after contact

In [ ]:
pfr_agg_rush['rushing_yards_before_contact_avg'] = pfr_agg_rush['rushing_yards_before_contact'] / pfr_agg_rush['carries']

In [ ]:
pfr_agg_rush['rushing_yards_after_contact_avg'] = pfr_agg_rush['rushing_yards_after_contact'] / pfr_agg_rush['carries']

In [ ]:
pfr_agg_rush['rushing_yards_before_contact_avg_allowed'] = pfr_agg_rush['rushing_yards_before_contact_allowed'] / pfr_agg_rush['carries_allowed']

In [ ]:
pfr_agg_rush['rushing_yards_after_contact_avg_allowed'] = pfr_agg_rush['rushing_yards_after_contact_allowed'] / pfr_agg_rush['carries_allowed']

In [ ]:
pfr_agg_rush.head()

In [ ]:
#Let's get our rolling averages

In [ ]:
original_week = pfr_agg_rush['week'].copy()

In [ ]:
columns_to_roll = [i for i in pfr_agg_rush.columns if i not in ['season', 'week', 'team']]

In [ ]:
columns_to_roll

In [ ]:
pfr_rush_rolling = pfr_agg_rush.groupby(['season', 'team'])[columns_to_roll].rolling(3).mean().reset_index()

In [ ]:
pfr_rush_rolling = pfr_rush_rolling.drop('level_2', axis=1)

In [ ]:
pfr_rush_rolling['week'] = original_week

In [ ]:
pfr_rush_rolling.head()

In [ ]:
#Lets confirm this works

In [ ]:
pfr_rush_rolling.iloc[:25, :]

In [ ]:
pfr_rush_rolling.columns = [i+'_rolling' if i not in ['season', 'team', 'week']\
                            else i for i in pfr_rush_rolling.columns]

In [ ]:
pfr_rush_rolling.head()

In [ ]:
#Lets join rolling values to our rush dataset

In [ ]:
len(pfr_agg_rush)

In [ ]:
len(pfr_agg_rush) == len(pfr_rush_rolling)

In [ ]:
pfr_agg_rush = pfr_agg_rush.merge(pfr_rush_rolling, on=['season', 'week', 'team'])

In [ ]:
len(pfr_agg_rush)

In [ ]:
#Lets rank these columns

In [ ]:
columns_to_rank = [i for i in pfr_agg_rush.columns if i not in ['season', 'week', 'team']]

In [ ]:
for col in columns_to_rank:
    rank_col_name = f'rank_{col}_weekly'
    pfr_agg_rush[rank_col_name] = pfr_agg_rush.groupby(['season', 'week'])[col].rank(ascending=False)

In [ ]:
pfr_agg_rush.head()

In [ ]:
#Lets get sums/avg up to each part of the season

In [ ]:
colums_to_expand_sum = ['carries',
 'rushing_yards_before_contact',
 'rushing_yards_after_contact',
 'rushing_broken_tackles',
 'carries_allowed',
 'rushing_yards_before_contact_allowed',
 'rushing_yards_after_contact_allowed',
 'rushing_broken_tackles_allowed']

In [ ]:
for col in colums_to_expand_sum:
    expand_col_name = f'expand_{col}'
    pfr_agg_rush[expand_col_name] = pfr_agg_rush.sort_values(by='week').groupby(['season', 'team'])[col].cumsum()

In [ ]:
pfr_agg_rush.head()

In [ ]:
#Lets check this is working

In [ ]:
pfr_agg_rush[['season', 'week', 'team',
              'carries', 'expand_carries', 
              'carries_allowed','expand_carries_allowed']].iloc[:25, :]

In [ ]:
#Great, lets do this for averages

In [ ]:
colums_to_expand_avg = ['rushing_yards_before_contact_avg',
 'rushing_yards_before_contact_avg_allowed',
 'rushing_yards_after_contact_avg_allowed',
 'rushing_yards_after_contact_avg']

In [ ]:
for col in colums_to_expand_avg:
    expand_col_name = f'expand_{col}'
    cumulative_sum = pfr_agg_rush.sort_values(by='week').groupby(['season', 'team'])[col].cumsum()
    cumulative_count = pfr_agg_rush.sort_values(by='week').groupby(['season', 'team'])[col].cumcount() + 1
    pfr_agg_rush[expand_col_name] = cumulative_sum / cumulative_count

In [ ]:
#Lets confirm this worked

In [ ]:
pfr_agg_rush[['season', 'week', 'team','rushing_yards_before_contact_avg',
              'expand_rushing_yards_before_contact_avg']].iloc[:25, :]

In [ ]:
#Now we'll get the rank of the expanding columns

In [ ]:
for col in pfr_agg_rush.columns:
    if 'expand_' in col:
        rank_col_name = f'rank_{col}_weekly'
        pfr_agg_rush[rank_col_name] = pfr_agg_rush.groupby(['season', 'week'])[col].rank(ascending=False)

In [ ]:
pfr_agg_rush.head()

In [ ]:
#Lets shift the data

In [ ]:
columns_to_shift = [i for i in pfr_agg_rush.columns if i not in ['season', 'team', 'week']]

In [ ]:
for i in columns_to_shift:
    pfr_agg_rush[i+'_shifted'] = pfr_agg_rush.groupby(['season', 'team'])[i].shift(periods=1)

In [ ]:
#Lets test this worked

In [ ]:
pfr_agg_rush[['season', 'team', 'week',
              'carries_allowed', 'carries_allowed_shifted',
             'carries_allowed_rolling', 'carries_allowed_rolling_shifted']].iloc[:25, :]

In [ ]:
#Let's keep the shifted data

In [ ]:
pfr_agg_rush = pfr_agg_rush[[i for i in pfr_agg_rush.columns if i in ['season', 'team', 'week']
                            or '_shifted' in i]]

In [ ]:
#Now let's join to weekly agg

In [ ]:
len(weekly_agg)

In [ ]:
len(weekly_agg) == len(pfr_agg_rush)

In [ ]:
len(pfr_agg_rush)

In [ ]:
weekly_agg = weekly_agg.merge(pfr_agg_rush, on=['season', 'week', 'team'])

In [ ]:
len(weekly_agg)

In [ ]:
weekly_agg.head()

# Injury Data

In [ ]:
#Lets get injury data

In [ ]:
inj = nfl.import_injuries(years)

In [ ]:
inj = inj[inj['game_type'] == 'REG'].reset_index(drop=True)

In [ ]:
inj.head()

In [ ]:
inj['team'].unique()

In [ ]:
#The table lists all players on a team, even if the players aren't injured

#Filter for those who are out
#These are injuries keeping players off the field

In [ ]:
inj['report_status'].unique()

In [ ]:
inj = inj[inj['report_status'] == 'Out'].reset_index(drop=True)

In [ ]:
#Changing raiders to LV

In [ ]:
inj.loc[inj['team'] == 'OAK', 'team'] = 'LV'

In [ ]:
inj.head()

In [ ]:
#Seaon should be an integer

In [ ]:
inj['season'] = inj['season'].astype('int')

In [ ]:
#Thinking for each team, each week, we get number of players injured

In [ ]:
inj_agg = inj.groupby(['season', 'week', 'team'])['gsis_id'].count().reset_index()

In [ ]:
inj_agg = inj_agg.rename(columns={'gsis_id':'players_injured'})

In [ ]:
inj_agg.head()

In [ ]:
print(len(inj_agg), len(weekly_agg))

In [ ]:
#These two won't equal because some teams won't have any injuries
#When joining, need to reset the column to be 0 if null

In [ ]:
weekly_agg = weekly_agg.merge(inj_agg, on=['season', 'week', 'team'], how='left')

In [ ]:
len(weekly_agg)

In [ ]:
weekly_agg['players_injured_adj'] = weekly_agg['players_injured'].apply(lambda x: 0 if pd.isna(x) else x)

In [ ]:
#Confirming the values have been adjusted so that null values show 0

weekly_agg.iloc[:32, -2:]

In [ ]:
#Looks good, we'll drop the inj column that has nulls

In [ ]:
weekly_agg = weekly_agg.drop('players_injured', axis=1)

# QBR Data

In [ ]:
#Get QBR data

In [ ]:
qbr = nfl.import_qbr(years, 'nfl', 'weekly')

In [ ]:
qbr = qbr[qbr['season_type'] == 'Regular'].reset_index(drop=True)

In [ ]:
qbr.head()

In [ ]:
#Changing raiders to LV
#Also in this table, LA rams are LAR, but in other tables they're just LA
#Same for washington

In [ ]:
qbr.loc[qbr['team_abb'] == 'OAK', 'team_abb'] = 'LV'
qbr.loc[qbr['opp_abb'] == 'OAK', 'opp_abb'] = 'LV'

qbr.loc[qbr['team_abb'] == 'LAR', 'team_abb'] = 'LA'
qbr.loc[qbr['opp_abb'] == 'LAR', 'opp_abb'] = 'LA'

qbr.loc[qbr['team_abb'] == 'WSH', 'team_abb'] = 'WAS'
qbr.loc[qbr['opp_abb'] == 'WSH', 'opp_abb'] = 'WAS'

In [ ]:
#Whats qualified??
#Are there rows w/errors?
qbr['qualified'].value_counts()

In [ ]:
#Some teams are repeated b/c they'll play multiple QBs in a game
#So we'll take an weighted avg of each team's metrics (QBR, epa_total, etc)

In [ ]:
qbr['share_of_plays'] = qbr['qb_plays'] / qbr.groupby(['season', 'team', 'game_week'])['qb_plays'].transform('sum')

In [ ]:
[i for i in qbr.iloc[:,9:19].columns]

In [ ]:
for i in qbr.iloc[:,9:19].columns:
    col_name = f'{i}_adj'
    qbr[col_name] = qbr[i] * qbr['share_of_plays']

In [ ]:
qbr[(qbr['game_week'] == 17)
   &
   (qbr['team'] == 'Dolphins')
   &
   (qbr['season'] == 2022)].iloc[:, 8:]

In [ ]:
#Lets see an example

In [ ]:
qbr[(qbr['game_week'] == 17)
   &
   (qbr['team'] == 'Dolphins')
   &
   (qbr['season'] == 2022)].groupby(['season', 'team', 'game_week'])[['qbr_total_adj',
                                                                    'qbr_raw_adj']].sum().reset_index()

In [ ]:
#This adjustment makes sense because not all QB play is equal
#A backup coming in during the 4th quarter when in the lead is different from a starter in the 1st quarter
#This being said, we'll grab an adj version and a maximum of each column
#This way we'll have the best QB performance and a weighted "team" qb performance

In [ ]:
#We'll pull both the weighted avg and max of all the metrics

In [ ]:
qbr_agg = qbr.groupby(['season', 'team_abb', 'game_week']).agg({'qbr_total':'max',
                                                            'qbr_total_adj':'sum',
                                                             'pts_added':'max',
                                                             'pts_added_adj':'sum',
                                                             'qb_plays':'max',
                                                             'qb_plays_adj':'sum',
                                                             'epa_total':'max',
                                                             'epa_total_adj':'sum',
                                                             'pass':'max',
                                                             'pass_adj':'sum',
                                                             'run':'max',
                                                             'run_adj':'sum',
                                                             'exp_sack':'max',
                                                             'exp_sack_adj':'sum',
                                                             'penalty':'max',
                                                             'penalty_adj':'sum',
                                                             'qbr_raw':'max',
                                                             'qbr_raw_adj':'sum',
                                                             'sack':'max',
                                                             'sack_adj':'sum'}).reset_index()

In [ ]:
qbr_agg.head()

In [ ]:
#Let's get QBR allowed columns now

In [ ]:
qbr_agg_def = qbr.groupby(['season', 'opp_abb', 'game_week']).agg({'qbr_total':'max',
                                                            'qbr_total_adj':'sum',
                                                             'pts_added':'max',
                                                             'pts_added_adj':'sum',
                                                             'qb_plays':'max',
                                                             'qb_plays_adj':'sum',
                                                             'epa_total':'max',
                                                             'epa_total_adj':'sum',
                                                             'pass':'max',
                                                             'pass_adj':'sum',
                                                             'run':'max',
                                                             'run_adj':'sum',
                                                             'exp_sack':'max',
                                                             'exp_sack_adj':'sum',
                                                             'penalty':'max',
                                                             'penalty_adj':'sum',
                                                             'qbr_raw':'max',
                                                             'qbr_raw_adj':'sum',
                                                             'sack':'max',
                                                             'sack_adj':'sum'}).reset_index()

In [ ]:
qbr_agg_def.columns = [i+'_allowed' if i not in ['season', 'team_abb', 'game_week', 'opp_abb']\
                      else i for i in qbr_agg.columns]

In [ ]:
qbr_agg_def.head()

In [ ]:
#Lets join the def and offensive qbr tables

In [ ]:
qbr_agg = qbr_agg.sort_values(by=['season','team_abb','game_week']).reset_index(drop=True)
qbr_agg_def = qbr_agg_def.sort_values(by=['season','team_abb','game_week']).reset_index(drop=True)

In [ ]:
qbr_agg = qbr_agg_def.merge(qbr_agg, on=['season','team_abb','game_week'], how='outer')

In [ ]:
#Sometimes QB play is so poor, that QBR stats aren't recorded/calculated
#In these instances, the team didn't have QBR, and the opposing team didn't have QBR allowed
#This means some teams aren't in the qbr table, requiring an outer join
#In these instances, we'll add a 0 for those rows

In [ ]:
qbr_agg = qbr_agg.sort_values(by=['season','team_abb','game_week']).reset_index(drop=True)

In [ ]:
qbr_agg.head()

In [ ]:
#Viewing the rows with null QBR values
#Again this is due to poor QB performance

In [ ]:
qbr_agg[qbr_agg['run'].isna()].iloc[:5, -20:]

In [ ]:
#QBR scales from 0 to 100
#To me, if your play was so bad a QBR couldn't be calculated, I'm giving you a 0
#This also means you added no expected points, so your epa should also be a 0
#Same logic for the other columns

In [ ]:
#Lets replace the null values with 0

qbr_agg = qbr_agg.fillna(0)

In [ ]:
#Confirming we don't have null values in the QBR values

qbr_agg[qbr_agg['run'].isna()].iloc[:5, -20:]

In [ ]:
qbr_agg.head()

In [ ]:
#Lets get a rolling avg of these metrics

In [ ]:
original_week = qbr_agg['game_week'].copy()

In [ ]:
columns_to_roll = [i for i in qbr_agg.columns if i not in ['season', 'team_abb', 'game_week']]

In [ ]:
qbr_rolling = qbr_agg.groupby(['season', 'team_abb'])[columns_to_roll].rolling(3).mean().reset_index()

In [ ]:
qbr_rolling = qbr_rolling.drop('level_2', axis=1)

In [ ]:
qbr_rolling['game_week'] = original_week

In [ ]:
qbr_rolling.head()

In [ ]:
len(qbr_rolling)

In [ ]:
#Confirming this is working

In [ ]:
qbr_rolling.iloc[14:25, :]

In [ ]:
#Lets add "rolling" to the column names

In [ ]:
qbr_rolling.columns = [i+'_rolling' if i not in ['season', 'team_abb', 'game_week'] \
                      else i for i in qbr_rolling.columns]

In [ ]:
#Lets join to qbr_agg

In [ ]:
len(qbr_agg)

In [ ]:
len(qbr_agg) == len(qbr_rolling)

In [ ]:
qbr_agg = qbr_agg.merge(qbr_rolling, on=['season', 'game_week', 'team_abb'], how='outer')

In [ ]:
qbr_agg.head()

In [ ]:
len(qbr_agg)

In [ ]:
#Lets get rankings of these columns

In [ ]:
columns_to_rank = [i for i in qbr_agg.columns if i not in ['season', 'game_week', 'team_abb']]

In [ ]:
columns_to_rank

In [ ]:
for col in columns_to_rank:
    rank_col_name = f'rank_{col}_weekly'
    qbr_agg[rank_col_name] = qbr_agg.groupby(['season', 'game_week'])[col].rank(ascending=False)

In [ ]:
qbr_agg.head()

In [ ]:
#Lets get season-long ranks
#These are indices, so the columns will be expanded based on their average over the season

In [ ]:
colums_to_expand_avg = ['qbr_total_allowed',
 'qbr_total_adj_allowed',
 'pts_added_allowed',
 'pts_added_adj_allowed',
 'qb_plays_allowed',
 'qb_plays_adj_allowed',
 'epa_total_allowed',
 'epa_total_adj_allowed',
 'pass_allowed',
 'pass_adj_allowed',
 'run_allowed',
 'run_adj_allowed',
 'exp_sack_allowed',
 'exp_sack_adj_allowed',
 'penalty_allowed',
 'penalty_adj_allowed',
 'qbr_raw_allowed',
 'qbr_raw_adj_allowed',
 'sack_allowed',
 'sack_adj_allowed',
 'qbr_total',
 'qbr_total_adj',
 'pts_added',
 'pts_added_adj',
 'qb_plays',
 'qb_plays_adj',
 'epa_total',
 'epa_total_adj',
 'pass',
 'pass_adj',
 'run',
 'run_adj',
 'exp_sack',
 'exp_sack_adj',
 'penalty',
 'penalty_adj',
 'qbr_raw',
 'qbr_raw_adj',
 'sack',
 'sack_adj',]

In [ ]:
for col in colums_to_expand_avg:
    expand_col_name = f'expand_{col}'
    cumulative_sum = qbr_agg.sort_values(by='game_week').groupby(['season', 'team_abb'])[col].cumsum()
    cumulative_count = qbr_agg.sort_values(by='game_week').groupby(['season', 'team_abb'])[col].cumcount() + 1
    qbr_agg[expand_col_name] = cumulative_sum / cumulative_count

In [ ]:
#Lets confirm this worked

In [ ]:
qbr_agg[['season', 'game_week', 'team_abb','qbr_total_allowed',
              'expand_qbr_total_allowed']].iloc[:25, :]

In [ ]:
#Now we'll get the rank of the expanding columns

In [ ]:
for col in qbr_agg.columns:
    if 'expand_' in col:
        rank_col_name = f'rank_{col}_weekly'
        qbr_agg[rank_col_name] = qbr_agg.groupby(['season', 'game_week'])[col].rank(ascending=False)

In [ ]:
qbr_agg.head()

In [ ]:
#Lets shift the columns

In [ ]:
columns_to_shift = [i for i in qbr_agg.columns if i not in ['season', 'game_week', 'team_abb']]

In [ ]:
for i in columns_to_shift:
    qbr_agg[i+'_shifted'] = qbr_agg.groupby(['season', 'team_abb'])[i].shift(periods=1)

In [ ]:
#Lets check this worked

In [ ]:
qbr_agg[['season', 'team_abb', 'game_week',
              'run', 'run_shifted',
             'run_rolling', 'run_rolling_shifted']].iloc[:25, :]

In [ ]:
#Lets keep the shifted data

In [ ]:
qbr_agg = qbr_agg[[i for i in qbr_agg.columns if i in ['season','team_abb', 'game_week']
                  or '_shifted' in i]]

In [ ]:
#Lets join to weekly_agg

In [ ]:
qbr_agg = qbr_agg.rename(columns={'game_week':'week',
                                'team_abb':'team'})

In [ ]:
len(weekly_agg)

In [ ]:
len(weekly_agg) == len(qbr_agg)

In [ ]:
weekly_agg = weekly_agg.merge(qbr_agg, on=['season', 'week', 'team'], how='left')

In [ ]:
weekly_agg.head()

# Game Info

In [ ]:
info = nfl.import_schedules(years)

In [ ]:
info = info[info['game_type'] == 'REG'].reset_index(drop=True)

In [ ]:
info.head()

In [ ]:
#Lets see some of the odds info

In [ ]:
info.iloc[:5, 24:32]

In [ ]:
info[~info['away_moneyline'].isna()]['season'].min()

In [ ]:
#Since we've had to change team names, lets confirm what teams in weekly agg aren't in info

In [ ]:
[i for i in info['away_team'].unique() if i not in weekly_agg['team'].unique()]

In [ ]:
#Changing raiders info

In [ ]:
info.loc[info['away_team'] == 'OAK', 'away_team'] = 'LV'
info.loc[info['home_team'] == 'OAK', 'home_team'] = 'LV'

In [ ]:
#Confirming that we have all the same teams now?

In [ ]:
[i for i in info['away_team'].unique() if i not in weekly_agg['team'].unique()]

In [ ]:
odds = info[['season', 'week', 'away_team', 'home_team', 'away_moneyline', 
      'home_moneyline', 'spread_line', 'away_spread_odds',
       'home_spread_odds', 'total_line', 'under_odds', 'over_odds',
            'home_score', 'away_score']]

In [ ]:
odds.head()

In [ ]:
#Comapring the spread_line to the historical odds site, spread_line is the away team spread
#So we can get home team spread by multiplying by -1 to get the inverse

In [ ]:
odds = odds.rename(columns={'spread_line':'away_spread'})

In [ ]:
odds['home_spread'] = odds['away_spread'] * -1

In [ ]:
odds.head()

In [ ]:
#Now lets get the score margin and compare to the betting spread

In [ ]:
odds['home_diff'] = odds['home_score'] - odds['away_score']
odds['away_diff'] = odds['away_score'] - odds['home_score']

In [ ]:
odds['home_cover'] = odds['home_spread'] + odds['home_diff']
odds['away_cover'] = odds['away_spread'] + odds['away_diff']

In [ ]:
#Lets also get the total points

In [ ]:
odds['total_score'] = odds['home_score'] + odds['away_score']

In [ ]:
#We can group by season, week, away team (away stats)
#Then we can group by season, week, home team (home stats)

#Then we can concat and change the column names to have a betting table

In [ ]:
odds.columns

In [ ]:
bet_away = odds[['season', 'week', 'away_team','away_moneyline',
                               'away_spread',
                               'away_spread_odds',
                                 'total_line',
                                 'under_odds',
                                 'over_odds',
                                 'away_cover',
                                 'home_cover',
                                  'away_score',
                                 'away_diff',
                                'home_score',
                                 'home_diff',
                                 'total_score']]

In [ ]:
bet_away = bet_away.rename(columns={'away_team':'team',
                                   'away_moneyline':'moneyline',
                                    'away_spread':'spread',
                                   'away_spread_odds':'spread_odds',
                                   'total_line':'exp_total_points',
                                   'away_cover':'cover',
                                    'home_cover':'cover_allowed',
                                   'away_score':'points_scored',
                                    'away_diff':'scoring_margin',
                                'home_score':'points_allowed',
                                 'home_diff':'scoring_margin_allowed',
                                   'total_score':'total_points_scored'})

In [ ]:
bet_away['is_home'] = 0

In [ ]:
bet_away.head()

In [ ]:
bet_home = odds[['season', 'week', 'home_team','home_moneyline',
                               'home_spread',
                               'home_spread_odds',
                                 'total_line',
                                 'under_odds',
                                 'over_odds',
                                 'home_cover',
                                 'away_cover',
                                  'home_score',
                                 'home_diff',
                                  'away_score',
                                 'away_diff',
                                 'total_score']]

In [ ]:
bet_home = bet_home.rename(columns={'home_team':'team',
                                   'home_moneyline':'moneyline',
                                    'home_spread':'spread',
                                   'home_spread_odds':'spread_odds',
                                   'total_line':'exp_total_points',
                                   'home_cover':'cover',
                                    'away_cover':'cover_allowed',
                                   'home_score':'points_scored',
                                    'home_diff':'scoring_margin',
                                  'away_score':'points_allowed',
                                 'away_diff':'scoring_margin_allowed',
                                   'total_score':'total_points_scored'})

In [ ]:
bet_home['is_home'] = 1

In [ ]:
bet_home.head()

In [ ]:
bet = pd.concat([bet_home, bet_away])

In [ ]:
bet = bet.sort_values(by=['season', 'team', 'week']).reset_index(drop=True)

In [ ]:
bet.index.is_unique

In [ ]:
bet.head()

In [ ]:
#Lets get rolling avg points scored, points allowed, scoring margin, margin allowed, spread, cover, cover allowed 
#Then we'll get these metrics up to that point in the season
#Then of course, we rank

In [ ]:
original_week = bet['week'].copy()

In [ ]:
bet_rolling = bet.groupby(['season', 'team'])[['points_scored', 
                                              'points_allowed',
                                              'scoring_margin',
                                              'scoring_margin_allowed', 
                                              'spread',
                                              'cover', 
                                              'cover_allowed']].rolling(3).mean().reset_index()

In [ ]:
bet_rolling = bet_rolling.drop('level_2', axis=1)

In [ ]:
bet_rolling['week'] = original_week

In [ ]:
#Lets change the column names

In [ ]:
bet_rolling.columns = [i+'_rolling' if i not in ['season', 'week', 'team'] else i for i in bet_rolling.columns]

In [ ]:
bet_rolling.iloc[:20]

In [ ]:
#Let's join to bet

In [ ]:
len(bet)

In [ ]:
len(bet) == len(bet_rolling)

In [ ]:
bet = bet.merge(bet_rolling, on=['season', 'team', 'week'])

In [ ]:
bet.head()

In [ ]:
#Lets get the metrics we rolled, but expanding across the season

In [ ]:
for col in ['points_scored', 
              'points_allowed',
              'scoring_margin',
              'scoring_margin_allowed', 
              'spread',
              'cover', 
              'cover_allowed']:
    expand_col_name = f'expand_{col}'
    bet[expand_col_name] = bet.sort_values(by='week').groupby(['season', 'team'])[col].cumsum()

In [ ]:
bet.head()

In [ ]:
#Now lets rank these columns

In [ ]:
for col in [i for i in bet.columns if 'expand_' in i or '_rolling' in i]:
    rank_col_name = f'rank_{col}_weekly'
    bet[rank_col_name] = bet.groupby(['season', 'week'])[col].rank(ascending=False)

In [ ]:
#Lets check this all works

In [ ]:
bet[bet['team'] == 'GB'][['season', 'week', 'points_scored', 'points_scored_rolling',
                          'expand_points_scored', 'rank_expand_points_scored_weekly',
                         'rank_points_scored_rolling_weekly']].iloc[:16]

In [ ]:
#Now let's shift all this data

In [ ]:
bet.columns

In [ ]:
for i in ['points_scored_rolling', 'points_allowed_rolling',
       'scoring_margin_rolling', 'scoring_margin_allowed_rolling',
       'spread_rolling', 'cover_rolling', 'cover_allowed_rolling',
       'expand_points_scored', 'expand_points_allowed',
       'expand_scoring_margin', 'expand_scoring_margin_allowed',
       'expand_spread', 'expand_cover', 'expand_cover_allowed',
       'rank_points_scored_rolling_weekly',
       'rank_points_allowed_rolling_weekly',
       'rank_scoring_margin_rolling_weekly',
       'rank_scoring_margin_allowed_rolling_weekly',
       'rank_spread_rolling_weekly', 'rank_cover_rolling_weekly',
       'rank_cover_allowed_rolling_weekly', 'rank_expand_points_scored_weekly',
       'rank_expand_points_allowed_weekly',
       'rank_expand_scoring_margin_weekly',
       'rank_expand_scoring_margin_allowed_weekly',
       'rank_expand_spread_weekly', 'rank_expand_cover_weekly',
       'rank_expand_cover_allowed_weekly']:
    bet[i+'_shifted'] = bet.groupby(['season', 'team'])[i].shift(periods=1)

In [ ]:
#Lets confirm

In [ ]:
bet[bet['season'] == 2018][['team', 'week', 'points_scored_rolling', 'points_allowed_rolling',
                           'points_scored_rolling_shifted', 'points_allowed_rolling_shifted']].iloc[:16]

In [ ]:
#Cool, now let's keep the shifted info

In [ ]:
bet = bet.drop(['points_scored_rolling', 'points_allowed_rolling',
       'scoring_margin_rolling', 'scoring_margin_allowed_rolling',
       'spread_rolling', 'cover_rolling', 'cover_allowed_rolling',
       'expand_points_scored', 'expand_points_allowed',
       'expand_scoring_margin', 'expand_scoring_margin_allowed',
       'expand_spread', 'expand_cover', 'expand_cover_allowed',
       'rank_points_scored_rolling_weekly',
       'rank_points_allowed_rolling_weekly',
       'rank_scoring_margin_rolling_weekly',
       'rank_scoring_margin_allowed_rolling_weekly',
       'rank_spread_rolling_weekly', 'rank_cover_rolling_weekly',
       'rank_cover_allowed_rolling_weekly', 'rank_expand_points_scored_weekly',
       'rank_expand_points_allowed_weekly',
       'rank_expand_scoring_margin_weekly',
       'rank_expand_scoring_margin_allowed_weekly',
       'rank_expand_spread_weekly', 'rank_expand_cover_weekly',
       'rank_expand_cover_allowed_weekly'], axis=1)

In [ ]:
bet.columns

In [ ]:
#Now we have a table w betting info
#This table also has items regarding stadium surface, division game, etc

In [ ]:
info.columns

In [ ]:
#Lets get this info but split between the away and home team

In [ ]:
info_away = info[['season', 'week', 'away_team', 'away_rest', 'overtime',
     'div_game', 'roof', 'surface']]

In [ ]:
info_away = info_away.rename(columns={'away_team':'team',
                                     'away_rest':'rest'})

In [ ]:
info_away.head()

In [ ]:
info_home = info[['season', 'week', 'home_team', 'home_rest', 'overtime',
     'div_game', 'roof', 'surface']]

In [ ]:
info_home = info_home.rename(columns={'home_team':'team',
                                     'home_rest':'rest'})

In [ ]:
info_home.head()

In [ ]:
#Since roof and surface are categorical, lets create dummy variables

In [ ]:
info_away_surface = pd.get_dummies(info_away['surface'])

In [ ]:
info_away_roof = pd.get_dummies(info_away['roof'])

In [ ]:
info_away = pd.concat([info_away, info_away_surface, info_away_roof], axis=1)

In [ ]:
info_home_surface = pd.get_dummies(info_home['surface'])

In [ ]:
info_home_roof = pd.get_dummies(info_home['roof'])

In [ ]:
info_home_roof.head()

In [ ]:
info_home = pd.concat([info_home, info_home_surface, info_home_roof], axis=1)

In [ ]:
info_home.head()

In [ ]:
#Lets combine the away and home info

In [ ]:
weekly_info = pd.concat([info_home, info_away])

In [ ]:
weekly_info = weekly_info.sort_values(by=['season', 'team', 'week']).reset_index(drop=True)

In [ ]:
#We dont need roof and surface columns since we got the dummy columns

In [ ]:
weekly_info = weekly_info.drop(['roof', 'surface'], axis=1)

In [ ]:
weekly_info.head()

In [ ]:
#Let's combine bet and weekly_info to weekly_agg

In [ ]:
bet.columns

In [ ]:
len(bet)

In [ ]:
len(bet) == len(weekly_agg)

In [ ]:
weekly_agg = weekly_agg.merge(bet, on=['season', 'week', 'team'])

In [ ]:
weekly_agg.head()

In [ ]:
#Lets join weekly info

In [ ]:
len(weekly_info)

In [ ]:
len(weekly_info) == len(weekly_agg)

In [ ]:
weekly_agg = weekly_agg.merge(weekly_info, on=['season', 'week', 'team'])

In [ ]:
#Now let's create some binary variables that we could use as result columns

In [ ]:
#First, lets make a column for whether a team covered the spread

In [ ]:
weekly_agg['is_cover'] = weekly_agg['cover'].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
#Lets check this worked using the GB/Det game from 2019 week 6

In [ ]:
weekly_agg[(weekly_agg['season'] == 2019)
    &(weekly_agg['week'] == 6)
    &((weekly_agg['team'] == 'DET')
      |(weekly_agg['team'] == 'GB'))][['season', 'week', 'team',
                             'spread', 'exp_total_points',
                             'cover', 'is_cover','points_scored','total_points_scored', 'is_home']]

In [ ]:
#Now let's get a column for whether a team won the game

In [ ]:
weekly_agg['is_winner'] = weekly_agg['scoring_margin'].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
#Now let's get a column for whether over under was hit

In [ ]:
weekly_agg['is_over'] = (weekly_agg['total_points_scored'] > weekly_agg['exp_total_points']).astype('int')

In [ ]:
#Now lets get whether they're favored

In [ ]:
weekly_agg['is_fav'] = np.where(weekly_agg['spread'] < 0, 1, 0)

In [ ]:
#Now let's get their "record" both ATS and Straight Up, as well as record for over/under and being favored

In [ ]:
#We can do this by getting an avg of is_cover, is_winner,and is_over, and is_fav

In [ ]:
for col in ['is_cover', 'is_winner', 'is_over', 'is_fav']:
    expand_col_name = f'{col}_record'
    cumulative_sum = weekly_agg.sort_values(by='week').groupby(['season', 'team'])[col].cumsum()
    cumulative_count = weekly_agg.sort_values(by='week').groupby(['season', 'team'])[col].cumcount() + 1
    weekly_agg[expand_col_name] = cumulative_sum / cumulative_count

In [ ]:
#Lets check that this worked

In [ ]:
weekly_agg[weekly_agg['team'] == 'GB'][['season', 'team', 'week', 
                                        'is_cover', 'is_cover_record',
                                        'is_winner', 'is_winner_record',
                                       'is_over', 'is_over_record',
                                       'is_fav', 'is_fav_record']].head(16)

In [ ]:
#Why not get a rank of these columns as well?

In [ ]:
for col in ['is_cover_record', 'is_winner_record', 'is_over_record', 'is_fav_record']:
    rank_col_name = f'rank_{col}'
    weekly_agg[rank_col_name] = weekly_agg.groupby(['season', 'week'])[col].rank(ascending=False)

In [ ]:
#Checking this worked

In [ ]:
weekly_agg[weekly_agg['team'] == 'GB'][['season', 'team', 'week', 
                                        'is_cover', 'is_cover_record', 'rank_is_cover_record',
                                        'is_winner', 'is_winner_record', 'rank_is_winner_record',
                                       'is_over', 'is_over_record', 'rank_is_over_record',
                                       'is_fav', 'is_fav_record', 'rank_is_fav_record'
                                       ]].head(16)

In [ ]:
#Great, now let's shift the record columns

In [ ]:
for i in ['is_cover_record', 'rank_is_cover_record','is_winner_record', 
          'rank_is_winner_record','is_over_record', 'rank_is_over_record',
         'is_fav_record', 'rank_is_fav_record']:
    weekly_agg[i+'_shifted'] = weekly_agg.groupby(['season', 'team'])[i].shift(periods=1)

In [ ]:
#Checking this

In [ ]:
weekly_agg[weekly_agg['team'] == 'GB'][['season', 'week',
            'is_cover_record_shifted', 'rank_is_cover_record_shifted','is_winner_record_shifted', 
          'rank_is_winner_record_shifted','is_over_record_shifted', 'rank_is_over_record_shifted',
                                       'is_fav_record_shifted', 'rank_is_fav_record_shifted']].head(20)

In [ ]:
#Cool, let's just keep the shifted columns

In [ ]:
weekly_agg = weekly_agg.drop(['is_cover_record', 'rank_is_cover_record','is_winner_record', 
          'rank_is_winner_record','is_over_record', 'rank_is_over_record',
                             'is_fav_record', 'rank_is_fav_record'], axis=1)

In [ ]:
#I realize that rushing carries were in weekly_data as well as in pfr_rush

In [ ]:
#Lets identify them

In [ ]:
[i for i in weekly_agg.columns if i.endswith(('_x', '_y'))]

In [ ]:
#Lets eliminate the y columns

In [ ]:
weekly_agg = weekly_agg.drop([i for i in weekly_agg.columns if i.endswith(('_y'))], axis=1)

In [ ]:
#Lets remove _x from the remaining columns

In [ ]:
weekly_agg.columns = [i for i in weekly_agg.columns.str.replace('_x', '')]

In [ ]:
[i for i in weekly_agg.columns if i.endswith(('_y'))]

In [ ]:
[i for i in weekly_agg.columns if i.endswith(('_x'))]

In [ ]:
#Confirming the columns that don't have shifted in the column name

In [ ]:
[i for i in weekly_agg.columns if not i.endswith('_shifted')]

In [ ]:
#We have our dataset!

In [ ]:
weekly_agg.head()

In [ ]:
#Lets save our data to a csv

In [ ]:
weekly_agg.to_csv('/Users/SwagMawi/Documents/nfl_training_data_2023.csv')